In [1]:
import pymc3 as pm

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
/home/ericmjl/anaconda/envs/fundl-dev/lib/python3.7/site-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7604 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


In [2]:
import theano.tensor as tt

In [3]:
from fundl.datasets import make_graph_counting_dataset
from fundl.utils import pad_graph
import numpy as onp
import networkx as nx
import jax.numpy as np
from chemgraph import atom_graph
import janitor.chemistry


In [4]:
import pandas as pd

df = (
    pd.read_csv("bace.csv")
    .rename_column("mol", "structure")
    .smiles2mol("structure", "mol")
    .join_apply(lambda x: atom_graph(x["mol"]), "graph")
    .join_apply(lambda x: len(x["graph"]), "graph_size")
)

Gs = df["graph"].tolist()

print("Generating feature matrices and adjacency matrices...")
Fs = []
As = []
for G in Gs:
    Fs.append(onp.vstack([d["features"] for n, d in G.nodes(data=True)]))
    As.append(onp.asarray(nx.adjacency_matrix(G).todense()))

largest_graph_size = max([len(G) for G in Gs])

print("Preparing outputs...")
# Next line is a dummy task, count number of nodes in graph.
# y = np.array([len(G) for G in Gs]).reshape(-1, 1)

# Next line is a real task.
y = df["pIC50"].values.reshape(-1, 1)

print("Padding graphs to correct size...")
for i, (F, A) in enumerate(zip(Fs, As)):
    F, A = pad_graph(F, A, largest_graph_size)
    Fs[i] = F
    As[i] = A


Generating feature matrices and adjacency matrices...
Preparing outputs...
Padding graphs to correct size...


In [6]:
y.min(), y.max()

(2.5445461, 10.522879)

In [7]:
Fs = onp.stack(Fs).astype(float)
As = onp.stack(As).astype(float)

print(Fs.shape)
print(As.shape)


(1513, 97, 9)
(1513, 97, 97)


In [9]:
from fundl.activations import relu

def dense_params(prefix, input_dim, output_dim):
    w = pm.Normal(f"{prefix}_w", mu=0, sigma=0.1, shape=(input_dim, output_dim))
    b = pm.Normal(f"{prefix}_b", mu=0, sigma=0.1, shape=(output_dim,))
    return dict(w=w, b=b)

def mpnn(params, A, F, nonlin=relu):
    """Follow semantics of fundl.layers.graph.mpnn"""
    F = tt.batched_dot(A, F)
    F = tt.dot(F, params["w"]) + params["b"]
    return nonlin(F)


def gather(F):
    """Follow semantics of fundl.layers.graph.gather"""
    return tt.sum(F, axis=1)

def dense(params, x, nonlin=relu):
    """Follow semantics of fundl.layers.dense"""
    a = nonlin(tt.dot(x, params["w"]) + params["b"])
    return a

In [21]:
with pm.Model() as model:
    # Priors on parameters.
    params = dict()
    params["graph1"] = dense_params("graph1", 9, 9)
    params["graph2"] = dense_params("graph2", 9, 5)
    params["dense1"] = dense_params("dense1", 5, 5)
    params["dense2"] = dense_params("dense2", 5, 1)
    
    acts1 = mpnn(params["graph1"], As, Fs)
    acts2 = mpnn(params["graph2"], As, acts1)
    out = gather(acts2)
    out = dense(params["dense1"], out)
    out = dense(params["dense2"], out)
    
    # Prior on noise in measurement.
    sd = pm.Exponential("sd", lam=1)
    
    # Likelihood
    like = pm.Normal("like", mu=out, sd=sd, observed=y)

In [ ]:
with model:
#     trace_nuts = pm.sample(2000, cores=1)
    approx = pm.fit(n=500000)
    trace = approx.sample(2000)


Average Loss = 4,075.4:  13%|█▎        | 63888/500000 [47:09<5:13:53, 23.16it/s]   

In [ ]:
trace["graph1_w"].mean(axis=0)

In [ ]:
with model:
    samples = pm.sample_posterior_predictive(trace)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(approx.hist)
plt.yscale("log")

In [ ]:
l, m, u = np.percentile(samples["like"], q=[25, 50, 75], axis=0)

ldiff = m - l
udiff = u - m

# plt.errorbar(
#     y, 
#     samples["like"].mean(axis=0),
#     yerr=[ldiff, udiff],
#     marker='o'
# )
plt.scatter(y, samples["like"].mean(axis=0), color='red')


plt.plot([y.min(), y.max()], [y.min(), y.max()])

Everything looks off by a factor of a few hundred, but sure, I think I can dig that.